In [33]:
#pip install pydataset
!pip install pydataset
from pydataset import data
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width',10000)

from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

In [34]:
df = pd.DataFrame(data('Fair'))
print(df.shape)
df.head()
                  

(601, 9)


,sex,age,ym,child,religious,education,occupation,rate,nbaffairs
1,male,37.0,10.00,no,3,18,7,4,0
2,female,27.0,4.00,no,4,14,6,4,0
3,female,32.0,15.00,yes,1,12,1,4,0
4,male,57.0,15.00,yes,5,18,6,5,0
5,male,22.0,0.75,no,2,17,6,3,0


In [35]:
df.loc[df.sex== 'male','sex']= 0
df.loc[df.sex== 'female', 'sex']=1

df['sex'] = df['sex'].astype(int)
   
df.loc[df.child== 'no', 'child'] = 0
df.loc[df.child=='yes','child'] = 1
df['child'] = df['child'].astype(int)

In [36]:
x=df[['religious', 'age', 'sex', 'ym','education','occupation','nbaffairs']]
y=df['rate']

In [37]:
#Create the baseline model 

regression=LinearRegression()
regression.fit(x,y)

first_model=(mean_squared_error(y_true=y, y_pred=regression.predict(x)))

In [38]:
print(first_model)

1.0498738644696668


In [39]:
coef_dict_baseline = {}

for coef, feat in zip(regression.coef_,x.columns):
        coef_dict_baseline[feat] = coef

coef_dict_baseline

{'religious': 0.042352811106391754,
 'age': -0.009059645428673812,
 'sex': 0.08882013337087144,
 'ym': -0.030458802565476582,
 'education': 0.06810255742293708,
 'occupation': -0.005979506852998105,
 'nbaffairs': -0.07882571247653965}

In [47]:
elastic=ElasticNet()

In [48]:
# Elastic Net Model

#Elastic net, just like ride and lasso regression, requires normalized data
#We also need to create a grid.



search=GridSearchCV(estimator=elastic,
                    param_grid={'alpha':np.logspace(-5,2,8),
                                'l1_ratio':[.2,.4,.6,.8]},
                    scoring='neg_mean_squared_error',
                    n_jobs=1,
                    refit=True,
                    cv=10)
search.fit(x,y)
search.best_params_

{'alpha': 0.1, 'l1_ratio': 0.8}

In [49]:
#These values are suitable here.

In [50]:
elastic=ElasticNet(alpha=0.001, l1_ratio=0.75)
elastic.fit(x,y)

second_model=(mean_squared_error(y_true=y, y_pred=elastic.predict(x)))
print(second_model)

1.0498780469772433


In [51]:
coef_dict_baseline = {}

for coef, feat in zip(elastic.coef_,x.columns):
    coef_dict_baseline[feat] = coef

coef_dict_baseline

{'religious': 0.041704711583328176,
 'age': -0.009134042982645958,
 'sex': 0.08468579772758185,
 'ym': -0.030312108487706442,
 'education': 0.06761018950736478,
 'occupation': -0.005910038772690869,
 'nbaffairs': -0.0788244729470811}

In [ ]:
#Coefficients mostly the same...but occupation is turning out to be 0
#We can remove this column from our data set.